In [ ]:
import requests
import tensorflow as tf

def predict_images(filenames, endpoint="http://localhost:8501/v1/models/rps_model:predict"):
    def images_preprocessing(filename):
        image = tf.io.decode_image(open(filename, 'rb').read(), channels=3)
        image = tf.image.resize(image, [150, 150])
        image = image / 255.0
        image_tensor = tf.expand_dims(image, 0)
        return image_tensor.numpy().tolist()

    map_labels = {0: "Bacterial wilt disease", 1: "Healthy", 2: "Manganese Toxicity"}
    predictions = []

    for filename in filenames:
        # Preprocess the image
        image_tensor = images_preprocessing(filename)
        json_data = {"instances": image_tensor}

        # Send the request to the Prediction API
        response = requests.post(endpoint, json=json_data)
        if response.status_code == 200 and 'predictions' in response.json():
            prediction = tf.argmax(response.json()['predictions'][0]).numpy()
            predictions.append((filename, map_labels[prediction]))
        else:
            predictions.append((filename, "Prediction failed"))

    return predictions

# Contoh penggunaan fungsi
filenames = [
    'images/Manganese_Toxicityt_disease_downscaled_original_IMG20230505172751 - Copy.jpg_1587ba61-c7b4-4d53-845a-61646ed31080.jpg',
    'images/Bacterial_wilt_disease_downscaled_original_20230506_131654.jpg_80e8f331-b7be-432e-b5a3-025359054c1c.jpg',
    'images/Healthy_disease_downscaled_original_20230506_123549.jpg_47d37696-509e-4861-8de9-bb2f2a4b89df.jpg',
    'images/Bacterial_wilt_disease_downscaled_original_20220228_154206_jpg.rf.6a32d848b2ca5e32570515e1a8373791.jpg_1e3462a1-7c1b-439a-8295-08d4bc683e9d.jpg',
    'images/Healthy_disease_downscaled_original_20230506_123011.jpg_42db2b63-31ba-4eda-ab8a-7a01f64ca22e.jpg',
    'images/Manganese_Toxicityt_disease_downscaled_original_IMG20230505172830.jpg_1155dc16-03fe-4f46-b218-552e634fad58.jpg',
    'images/bacterial_disease_1.jpg',
    'images/healthy_1.jpg'
]

results = predict_images(filenames)

for filename, prediction in results:
    print(f"File: {filename}")
    print(f"Predicted Class: {prediction}")
